In [1]:
import numpy as np
import tensorflow as tf
import pickle
import random
from matplotlib import pyplot as plt
import math
from PIL import Image
from matplotlib.pyplot import imshow
import h5py
import gc

In [ ]:
hdf_file = 'datasets/pickles/SVHN_multi.hdf5'

hdf = h5py.File(hdf_file,'r')
svhn_test_dataset = hdf['test_images'][:]
svhn_test_labels = hdf['test_labels'][:]
svhn_train_dataset = hdf['train_images'][:]
svhn_train_labels = hdf['train_labels'][:]
svhn_valid_dataset = hdf['valid_images'][:]
svhn_valid_labels = hdf['valid_labels'][:]

hdf.close()    

print(svhn_train_dataset.shape, svhn_train_labels.shape)
print(svhn_test_dataset.shape, svhn_test_labels.shape)
print(svhn_valid_dataset.shape, svhn_valid_labels.shape)

In [ ]:
svhn_train_dataset = svhn_train_dataset.astype(np.float32)
svhn_test_dataset = svhn_test_dataset.astype(np.float32)
svhn_valid_dataset = svhn_valid_dataset.astype(np.float32)

svhn_train_labels = svhn_train_labels.astype(np.int32)
svhn_test_labels = svhn_test_labels.astype(np.int32)
svhn_valid_labels = svhn_valid_labels.astype(np.int32)

In [2]:
hdf_file = 'datasets/pickles/box/SVHN_multi_box.hdf5'

hdf = h5py.File(hdf_file,'r')
svhn_train_box_dataset = hdf['train_images'][:]
svhn_train_box_labels = hdf['train_labels'][:]
svhn_test_box_dataset = hdf['test_images'][:]
svhn_test_box_labels = hdf['test_labels'][:]
svhn_valid_box_dataset = hdf['valid_images'][:]
svhn_valid_box_labels = hdf['valid_labels'][:]
            
hdf.close()    

print(svhn_train_box_dataset.shape, svhn_train_box_labels.shape)
print(svhn_test_box_dataset.shape, svhn_test_box_labels.shape)
print(svhn_valid_box_dataset.shape, svhn_valid_box_labels.shape)

(230069, 32, 96, 1) (230069, 6)
(13068, 32, 96, 1) (13068, 6)
(5684, 32, 96, 1) (5684, 6)


In [5]:
svhn_train_box_dataset = svhn_train_box_dataset.astype(np.float32)
svhn_test_box_dataset = svhn_test_box_dataset.astype(np.float32)
svhn_valid_box_dataset = svhn_valid_box_dataset.astype(np.float32)

svhn_train_box_labels = svhn_train_box_labels.astype(np.int32)
svhn_test_box_labels = svhn_test_box_labels.astype(np.int32)
svhn_valid_box_labels = svhn_valid_box_labels.astype(np.int32)

In [ ]:
print(svhn_train_dataset.shape, svhn_train_labels.shape)
print(svhn_test_dataset.shape, svhn_test_labels.shape)
print(svhn_valid_dataset.shape, svhn_valid_labels.shape)

In [6]:
def acc(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 2).T == labels) / predictions.shape[1] / predictions.shape[0])

In [7]:
def randomize_dataset(images, labels):
    shuffle = list(zip(images, labels))
    np.random.shuffle(shuffle)
    i, l = zip(*shuffle)
    i, l = np.asarray(i), np.asarray(l)
    return i, l

In [ ]:
svhn_train_dataset, svhn_train_labels = randomize_dataset(svhn_train_dataset, svhn_train_labels)
svhn_test_dataset, svhn_test_labels = randomize_dataset(svhn_test_dataset, svhn_test_labels)
svhn_valid_dataset, svhn_valid_labels = randomize_dataset(svhn_valid_dataset, svhn_valid_labels)

In [ ]:
svhn_train_dataset_1 = svhn_train_dataset[0:125000]
svhn_train_labels_1 = svhn_train_labels[0:125000]

print(svhn_train_dataset_1.shape, svhn_train_labels_1.shape)
print(svhn_train_dataset.shape, svhn_train_labels.shape)
print(svhn_test_dataset.shape, svhn_test_labels.shape)
print(svhn_valid_dataset.shape, svhn_valid_labels.shape)

In [8]:
graph_svhn = tf.Graph()

with graph_svhn.as_default():
    HEIGHT = 32
    WIDTH = 32*3
    
    X = tf.placeholder(tf.float32, [None, HEIGHT, WIDTH, 1])
    Y_ = tf.placeholder(tf.int32, [None, 6])
    
    # Learning Rate - alpha
    alpha = tf.placeholder(tf.float32)
    
    # Dropout Probablity
    pkeep = tf.placeholder(tf.float32)
    
    # 5 Layers and their no of neurons
    # 3 Convolutional Layers and a fully connected layer
    K = 6     # First Conv Layer with depth 6
    L = 12     # Second Conv Layer with depth 12
    M = 24    # Third Conv layer with depth 24
    N = 200   # Fourth Fully Connected layer with 200 neurons
    # Last one will be softmax layer with 10 output channels
    
    W1 = tf.Variable(tf.truncated_normal([6, 6, 1, K], stddev=0.1))    # 6x6 patch, 1 input channel, K output channels
    B1 = tf.Variable(tf.constant(0.1, tf.float32, [K]))
    
    W2 = tf.Variable(tf.truncated_normal([5, 5, K, L], stddev=0.1))
    B2 = tf.Variable(tf.constant(0.1, tf.float32, [L]))
    
    W3 = tf.Variable(tf.truncated_normal([4, 4, L, M], stddev=0.1))
    B3 = tf.Variable(tf.constant(0.1, tf.float32, [M]))
    
    W5_1 = tf.Variable(tf.truncated_normal([N, 11], stddev=0.1))
    B5_1 = tf.Variable(tf.constant(0.1, tf.float32, [11]))
    
    W5_2 = tf.Variable(tf.truncated_normal([N, 11], stddev=0.1))
    B5_2 = tf.Variable(tf.constant(0.1, tf.float32, [11]))
    
    W5_3 = tf.Variable(tf.truncated_normal([N, 11], stddev=0.1))
    B5_3 = tf.Variable(tf.constant(0.1, tf.float32, [11]))
    
    W5_4 = tf.Variable(tf.truncated_normal([N, 11], stddev=0.1))
    B5_4 = tf.Variable(tf.constant(0.1, tf.float32, [11]))
    
    W5_5 = tf.Variable(tf.truncated_normal([N, 11], stddev=0.1))
    B5_5 = tf.Variable(tf.constant(0.1, tf.float32, [11]))
    
    # Model
    stride = 1  # output is 32x96
    Y1 = tf.nn.relu(tf.nn.conv2d(X, W1, strides=[1, stride, stride, 1], padding='SAME') + B1)
    
    stride = 2  # output is 16x48
    Y2 = tf.nn.relu(tf.nn.conv2d(Y1, W2, strides=[1, stride, stride, 1], padding='SAME') + B2)
    
    stride = 2  # output is 8x24
    Y3 = tf.nn.relu(tf.nn.conv2d(Y2, W3, strides=[1, stride, stride, 1], padding='SAME') + B3)

    # reshape the output from the third convolution for the fully connected layer
    shape = Y3.get_shape().as_list()
    YY = tf.reshape(Y3, shape=[-1, shape[1] * shape[2] * shape[3]])
    
    W4 = tf.Variable(tf.truncated_normal([shape[1] * shape[2] * shape[3], N], stddev=0.1))
    B4 = tf.Variable(tf.constant(0.1, tf.float32, [N]))

    Y4 = tf.sigmoid(tf.matmul(YY, W4) + B4)
    YY4 = tf.nn.dropout(Y4, pkeep)
    
    Ylogits_1 = tf.matmul(YY4, W5_1) + B5_1
    Ylogits_2 = tf.matmul(YY4, W5_2) + B5_2
    Ylogits_3 = tf.matmul(YY4, W5_3) + B5_3
    Ylogits_4 = tf.matmul(YY4, W5_4) + B5_4
    Ylogits_5 = tf.matmul(YY4, W5_5) + B5_5   
    ## ('Ylogits_1 shape : ', [None, 11])
    
    Y_1 = tf.nn.softmax(Ylogits_1)
    Y_2 = tf.nn.softmax(Ylogits_2)
    Y_3 = tf.nn.softmax(Ylogits_3)
    Y_4 = tf.nn.softmax(Ylogits_4)
    Y_5 = tf.nn.softmax(Ylogits_5)
   
    cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_1, Y_[:,1])) +\
    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_2, Y_[:,2])) +\
    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_3, Y_[:,3])) +\
    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_4, Y_[:,4])) +\
    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_5, Y_[:,5]))

    train_prediction = tf.pack([Y_1, Y_2, Y_3, Y_4, Y_5])
    
    train_step = tf.train.AdamOptimizer(alpha).minimize(cross_entropy)
    
    W_s = tf.pack([tf.reduce_max(tf.abs(W1)),tf.reduce_max(tf.abs(W2)),tf.reduce_max(tf.abs(W3)),tf.reduce_max(tf.abs(W4))])
    b_s = tf.pack([tf.reduce_max(tf.abs(B1)),tf.reduce_max(tf.abs(B2)),tf.reduce_max(tf.abs(B3)),tf.reduce_max(tf.abs(B4))])
    
    
    model_saver = tf.train.Saver()

In [ ]:
train_data = svhn_train_dataset_1
label_data = svhn_train_labels_1
print('train : ', train_data.shape, '  test : ', label_data.shape)

num_steps_1 = 10001
batch_size = 128

with tf.Session(graph=graph_svhn) as session:
#     tf.initialize_all_variables().run() 
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps_1):
        #  learning rate decay
        max_learning_rate = 0.0005
        min_learning_rate = 0.0001
#         max_learning_rate = 0.000
#         min_learning_rate = 0.000
        decay_speed = 5000.0
        learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-step/decay_speed)
#         print(svhn_train_labels.shape[0])
        offset = (step * batch_size) % (label_data.shape[0] - batch_size)
        batch_data = train_data[offset:(offset + batch_size), :, :, :]
        batch_labels = label_data[offset:(offset + batch_size), :]
        
        feed_dict = {X : batch_data, Y_ : batch_labels, pkeep : 0.80, alpha : learning_rate}
#         _, l, train_pred = session.run([train_step, cross_entropy, train_prediction], feed_dict=feed_dict)
        t_step, l, train_pred, W, b = session.run([train_step, cross_entropy, train_prediction, W_s, b_s], feed_dict=feed_dict)

#         if(step > 1520 and step < 1526) :
#             if(step % 1 == 0):
#                 print('~~~~~~~~~~~~~~~~~~~~~~~')
#                 print('W : ', W)
#                 print('b : ', b)
#                 print('loss : ', l)
#                 print('t_s : ', t_step)
#                 print('pred : ', np.argmax(train_pred))
#                 print('~~~~~~~~~~~~~~~~~~~~~~~')
        
#         if (step % 100 == 0):
#             print('W : ', W)
#             print('b : ', b)
#             print('loss : ', l)
#             print('pred : ', np.argmax(train_pred))
#             print('       ')
    
        if (step % 500 == 0):
            print('W : ', W)
            print('b : ', b)
            print('Loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % acc(train_pred, batch_labels[:,1:6]))
            print('Learning rate : ', learning_rate)
            print('    ')
            
    print('Training Complete on SVHN Data')
    
    save_path = model_saver.save(session, "saved_models/CNN_SVHN_New.ckpt")
    print("Model saved in file: %s" % save_path)

In [11]:
train_data = svhn_train_box_dataset
label_data = svhn_train_box_labels
print('train : ', train_data.shape, '  test : ', label_data.shape)

num_steps_1 = 50001
batch_size = 128

with tf.Session(graph=graph_svhn) as session:
#     tf.initialize_all_variables().run() 
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps_1):
        #  learning rate decay
        max_learning_rate = 0.0005
        min_learning_rate = 0.0001
#         max_learning_rate = 0.000
#         min_learning_rate = 0.000
        decay_speed = 5000.0
        learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-step/decay_speed)
#         print(svhn_train_labels.shape[0])
        offset = (step * batch_size) % (label_data.shape[0] - batch_size)
        batch_data = train_data[offset:(offset + batch_size), :, :, :]
        batch_labels = label_data[offset:(offset + batch_size), :]
        
        feed_dict = {X : batch_data, Y_ : batch_labels, pkeep : 0.80, alpha : learning_rate}
#         _, l, train_pred = session.run([train_step, cross_entropy, train_prediction], feed_dict=feed_dict)
        t_step, l, train_pred, W, b = session.run([train_step, cross_entropy, train_prediction, W_s, b_s], feed_dict=feed_dict)
    
        if (step % 500 == 0):
            print('W : ', W)
            print('b : ', b)
            print('Loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % acc(train_pred, batch_labels[:,1:6]))
            print('Learning rate : ', learning_rate)
            print('    ')
            
    print('Training Complete on SVHN Data')
    
    save_path = model_saver.save(session, "saved_models/box/CNN_SVHN_Box.ckpt")
    print("Model saved in file: %s" % save_path)

train :  (230069, 32, 96, 1)   test :  (230069, 6)
Initialized
W :  [ 0.19074975  0.1999169   0.19990675  0.19999984]
b :  [ 0.1  0.1  0.1  0.1]
Loss at step 0: 11.967587
Minibatch accuracy: 8.4%
Learning rate :  0.0005
    
W :  [ 0.25407082  0.27844286  0.30393046  0.30161068]
b :  [ 0.14115366  0.11585545  0.1124301   0.11016332]
Loss at step 500: 4.559722
Minibatch accuracy: 71.7%
Learning rate :  0.00046193496721438383
    
W :  [ 0.26474366  0.34329158  0.312334    0.33900106]
b :  [ 0.19042931  0.12220962  0.11910044  0.11027805]
Loss at step 1000: 3.211305
Minibatch accuracy: 82.0%
Learning rate :  0.00042749230123119273
    
W :  [ 0.26771152  0.37546405  0.32671165  0.3927843 ]
b :  [ 0.22279507  0.12939571  0.13068403  0.10978457]
Loss at step 1500: 2.812945
Minibatch accuracy: 83.8%
Learning rate :  0.00039632728827268716
    
W :  [ 0.27037984  0.3821643   0.33027631  0.425749  ]
b :  [ 0.25708118  0.13420933  0.14712055  0.11008257]
Loss at step 2000: 2.288783
Minibatch a

In [ ]:
with tf.Session(graph=graph_svhn) as session:
    model_saver.restore(session, "saved_models/CNN_SVHN_New.ckpt")
    print("Model restored.") 
    print('Initialized')
    
    _, l, predictions = session.run([train_step, cross_entropy, train_prediction], feed_dict={X : svhn_test_dataset[:500], Y_ : svhn_test_labels[:500], pkeep : 1.0, alpha : 0.002})
    print('Test accuracy: ', acc(predictions, svhn_test_labels[:500,1:6]))
    
    del l, predictions
    _, l, predictions = session.run([train_step, cross_entropy, train_prediction], feed_dict={X : svhn_valid_dataset, Y_ : svhn_valid_labels, pkeep : 1.0, alpha : 0.002})
    print('Validation accuracy: ', acc(predictions, svhn_valid_labels[:,1:6]))

In [12]:
with tf.Session(graph=graph_svhn) as session:
    model_saver.restore(session, "saved_models/box/CNN_SVHN_Box.ckpt")
    print("Model restored.") 
    print('Initialized')
    
    t_batch = 1000
    
    test_no = int(svhn_test_box_labels.shape[0] / t_batch)
    for i in range(test_no - 1):
        data = svhn_test_box_dataset[i*t_batch:(i+1)*t_batch]
        labels = svhn_test_box_labels[i*t_batch:(i+1)*t_batch]
        
        print('Test-Accuracy', ' i : ', i)
        _, l, predictions = session.run([train_step, cross_entropy, train_prediction], feed_dict={X : data, Y_ : labels, pkeep : 1.0, alpha : 0.002})
        print('Test accuracy: ', acc(predictions, labels[:,1:6]))
        print('       ')
    
    print('-----VALIDIDATION------')
    valid_no = int(svhn_valid_box_labels.shape[0] /  t_batch)
    for i in range(valid_no - 1):
        data = svhn_valid_box_dataset[i*t_batch:(i+1)*t_batch]
        labels = svhn_valid_box_labels[i*t_batch:(i+1)*t_batch]
        
        print('Valid-Accuracy', ' i : ', i)
        _, l, predictions = session.run([train_step, cross_entropy, train_prediction], feed_dict={X : data, Y_ : labels, pkeep : 1.0, alpha : 0.002})
        print('Valid accuracy: ', acc(predictions, labels[:,1:6]))
        print('-------')
        
#     _, l, predictions = session.run([train_step, cross_entropy, train_prediction], feed_dict={X : svhn_valid_box_dataset, Y_ : svhn_valid_box_labels, pkeep : 1.0, alpha : 0.002})
#     print('Validation accuracy: ', acc(predictions, svhn_valid_box_labels[:,1:6]))


Model restored.
Initialized
Test-Accuracy  i :  0
Test accuracy:  93.84
       
Test-Accuracy  i :  1
Test accuracy:  94.0
       
Test-Accuracy  i :  2
Test accuracy:  93.78
       
Test-Accuracy  i :  3
Test accuracy:  93.68
       
Test-Accuracy  i :  4
Test accuracy:  94.0
       
Test-Accuracy  i :  5
Test accuracy:  93.66
       
Test-Accuracy  i :  6
Test accuracy:  93.4
       
Test-Accuracy  i :  7
Test accuracy:  92.84
       
Test-Accuracy  i :  8
Test accuracy:  93.38
       
Test-Accuracy  i :  9
Test accuracy:  93.1
       
Test-Accuracy  i :  10
Test accuracy:  93.38
       
Test-Accuracy  i :  11
Test accuracy:  93.4
       
-----VALIDIDATION------
Valid-Accuracy  i :  0
Valid accuracy:  93.96
-------
Valid-Accuracy  i :  1
Valid accuracy:  94.12
-------
Valid-Accuracy  i :  2
Valid accuracy:  91.88
-------
Valid-Accuracy  i :  3
Valid accuracy:  92.42
-------
